In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [11]:
from pathlib import Path
import numpy as np
import pandas as pd
from neurodsp.filt import filter_signal
from ecephys.signal.csd import kcsd_npix

In [4]:
from ecephys.graham import paths, channel_groups
from ecephys.sglx_utils import load_timeseries
from ecephys.scoring import load_visbrain_hypnogram
from ecephys.plot import plot_timeseries_interactive

In [5]:
condition = "SR"
subject = "Doppio"

## Load the data

In [8]:
sr_chans = channel_groups.stratum_radiatum_csd[subject]
so_chans = channel_groups.stratum_oriens_100um[subject]
hpc_chans = channel_groups.hippocampus[subject]
bin_path = Path(paths.lfp_bin[condition][subject])

In [25]:
(time, sr_lfps, fs) = load_timeseries(bin_path, sr_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


In [26]:
(time, so_lfps, fs) = load_timeseries(bin_path, so_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


In [9]:
(time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


## Explore LFPs

In [13]:
plot_timeseries_interactive(time, sr_lfps, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [12]:
lfps = (sr_lfps.T - so_lfps.T).T

In [14]:
plot_timeseries_interactive(time, lfps, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Filter signal

In [31]:
f_range = (None, 35)
filtered_sr_lfps = filter_signal(sr_lfps.T, fs, 'lowpass', f_range)

In [32]:
plot_timeseries_interactive(time, filtered_sr_lfps.T, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [47]:
f_range = (1, 35)
filtered_sr_lfps = filter_signal(sr_lfps.T, fs, 'bandpass', f_range)

In [53]:
plot_timeseries_interactive(time, filtered_sr_lfps.T, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Detection type I: Thresholding LFP negativity by Z-score

In [35]:
filtered_sr_lfps.shape

(13, 18000019)

In [36]:
time = np.arange(0, filtered_sr_lfps.shape[1], 1)
time = time / fs
time.shape

(18000019,)

In [37]:
from ecephys.signal.ripples import _threshold_by_zscore
from ripple_detection.core import get_envelope

filtered_lfps = -filtered_sr_lfps.T
not_null = np.all(pd.notnull(filtered_lfps), axis=1)
filtered_lfps, _time = (filtered_lfps[not_null], time[not_null])
combined_filtered_lfps = np.sum(filtered_lfps, axis=1)

candidate_spw_times = _threshold_by_zscore(
    combined_filtered_lfps,
    _time,
    minimum_duration=0.030,
    detection_zscore_threshold=3,
    boundary_zscore_threshold=1,
)

index = pd.Index(np.arange(len(candidate_spw_times)) + 1, name="spw_number")
spw_times = pd.DataFrame(candidate_spw_times, columns=["start_time", "end_time"], index=index)

In [40]:
from ecephys.signal.ripples import compute_ripple_features

spw_times = compute_ripple_features(_time, filtered_lfps, spw_times, fs, 'Kay', smoothing_sigma=0.0)

In [41]:
spw_times

,start_time,end_time,duration,center_time,nadir_time,envelope_integral,envelope_peak,mean_rms,summed_rms,max_rms,mean_amplitude,max_amplitude
spw_number,,,,,,,,,,,,
1,87.180712,87.341112,0.1604,87.260912,87.245112,2.074016e+06,6038.730307,1127.940209,14663.222714,1391.427326,1357.071843,1587.116379
2,111.071488,111.267088,0.1956,111.169288,111.131888,2.137053e+06,5199.039163,980.637062,12748.281806,1232.776487,1145.277519,1413.523050
3,187.069812,187.143012,0.0732,187.106412,187.102612,8.543671e+05,5060.473777,1059.737657,13776.589536,1376.463334,1134.135785,1435.018190
4,195.683803,195.804603,0.1208,195.744203,195.729803,1.814641e+06,6817.911495,1295.059545,16835.774080,1629.288390,1675.378367,1995.634152
5,240.159358,240.225758,0.0664,240.192558,240.178958,8.621300e+05,6490.307593,1212.907491,15767.797384,1575.556100,1323.835739,1668.590650
...,...,...,...,...,...,...,...,...,...,...,...,...
659,7153.541206,7153.623206,0.0820,7153.582206,7153.562006,1.064825e+06,5921.576482,1139.380738,14811.949593,1352.435056,1310.604909,1587.718956
660,7177.377182,7177.452382,0.0752,7177.414782,7177.407582,9.330040e+05,5988.379570,1141.422545,14838.493089,1507.239235,1265.736957,1608.657031
661,7187.976372,7188.066772,0.0904,7188.021572,7188.021572,1.004013e+06,5271.781748,1022.334615,13290.349997,1276.595171,940.310951,1085.543914


In [44]:
from ecephys.plot import plot_ripple

from ipywidgets import (
    fixed,
    interact,
)

_, ax = plt.subplots(3, 1, figsize=(18, 6))
_ = interact(
    plot_ripple,
    time=fixed(time),
    lfps=fixed(sr_lfps),
    filtered_lfps=fixed(filtered_sr_lfps.T),
    fs=fixed(fs),
    ripple_times=fixed(spw_times),
    window_length=(0.25, 2, 0.25),
    ripple_number=(1, len(spw_times), 1),
    ax=fixed(ax),
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='window_length', max=2.0, min=0.25, step=0.25), IntSl…

## Detection type II: CSD

In [16]:
hpc_csd, estm_x = kcsd_npix(hpc_lfps)

In [17]:
plot_timeseries_interactive(time, hpc_csd.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [23]:
from ripple_detection.core import gaussian_smooth

smoothed_hpc_csd = gaussian_smooth(hpc_csd.T, 0.002, fs)

In [24]:
plot_timeseries_interactive(time, smoothed_hpc_csd)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [12]:
f_range = (None, 20)
filtered_hpc_lfps = filter_signal(hpc_lfps.T, fs, 'lowpass', f_range)

In [13]:
filtered_hpc_csd, estm_x = kcsd_npix(filtered_hpc_lfps.T)

In [15]:
plot_timeseries_interactive(time, filtered_hpc_csd.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()